In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500

In [2]:
import os

In [3]:
from pycountry import countries

In [4]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/jupyter_notebooks'

In [5]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [6]:
from hopkins_cleaner import HopkinsDataFull

In [7]:
self = HopkinsDataFull()

In [8]:
self.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/us_data_cleaner.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data = pd.concat([df1, df2], axis=0)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)


In [ ]:
df = self.confirmed.JH_raw

In [ ]:
df = df.loc[df['province_or_state'].fillna("NAH").str.contains('OR')]

In [ ]:
df = df["province_or_state"].str.split(", ", expand=True)

In [ ]:
df[1].str.strip()

In [9]:
df = self.data

In [10]:
df.loc[df['province_or_state'].str.contains('OR')]

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
12308,US,2020-01-22,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12309,US,2020-01-23,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12310,US,2020-01-24,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12311,US,2020-01-25,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12312,US,2020-01-26,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12313,US,2020-01-27,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12314,US,2020-01-28,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12315,US,2020-01-29,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12316,US,2020-01-30,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1
12317,US,2020-01-31,OR,0,OR-US,0.0,0.0,0,0.0,0.0,...,0.0,Oregon,44.0,-120.5,NaN,NaN,US,USA,327167434.0,38.1


In [ ]:
df.loc[df['state_name'] == 'Oregon'][['state_and_country','date','first_case_state_rank','first_case_country_rank']]

In [ ]:
df.loc[df['country_code_2'] == 'US']

In [11]:
df.loc[df['state_name'] == 'Virginia']

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
12788,US,2020-01-22,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12789,US,2020-01-23,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12790,US,2020-01-24,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12791,US,2020-01-25,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12792,US,2020-01-26,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12793,US,2020-01-27,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12794,US,2020-01-28,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12795,US,2020-01-29,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12796,US,2020-01-30,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1
12797,US,2020-01-31,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1


In [ ]:
pwd

In [ ]:
cd 

In [12]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

#### Make sure all US states have the same number of days 

In [18]:
us = df.loc[df['country_or_region'] == 'US']

In [19]:
us.groupby(['province_or_state'])['daily_new_cases'].size().reset_index()

,province_or_state,daily_new_cases
0,AK,53
1,AL,53
2,AR,53
3,AZ,53
4,CA,53
5,CO,53
6,CT,53
7,DC,53
8,DE,53
9,Diamond Princess,53


#### Total US cases 

In [20]:
us['daily_new_cases'].sum()

2727.0

In [24]:
df.head(2)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
0,Thailand,2020-01-22,Not Provided,2,Not Provided-Thailand,0.0,2.0,0,0.0,0.0,...,0.0,NaN,15.0,101.0,NaN,NaN,TH,THA,69428524.0,37.7
1,Thailand,2020-01-23,Not Provided,3,Not Provided-Thailand,2.0,1.0,0,0.0,0.0,...,0.0,NaN,15.0,101.0,NaN,NaN,TH,THA,69428524.0,37.7


In [22]:
max_date = df['date'].max()

In [25]:
yesterday = df.loc[df['date'] == max_date]
yesterday['running_total_cases'].sum(), yesterday['running_total_recoveries'].sum(), yesterday['running_total_deaths'].sum()

(156102, 72624, 5819)

In [26]:
df['daily_new_cases'].sum(), df['daily_new_recoveries'].sum(), df['daily_new_deaths'].sum()

(156102.0, 72624.0, 5819.0)

In [28]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
13336,US,2020-03-10,WA,267,WA-US,122.0,145.0,1,1.0,0.0,...,4.0,Washington,47.751076,-120.740135,8.0,16.0,US,USA,327167434.0,38.1
13337,US,2020-03-11,WA,366,WA-US,267.0,99.0,1,1.0,0.0,...,6.0,Washington,47.751076,-120.740135,9.0,17.0,US,USA,327167434.0,38.1
13338,US,2020-03-12,WA,442,WA-US,366.0,76.0,1,1.0,0.0,...,2.0,Washington,47.751076,-120.740135,10.0,18.0,US,USA,327167434.0,38.1
13339,US,2020-03-13,WA,568,WA-US,442.0,126.0,1,1.0,0.0,...,6.0,Washington,47.751076,-120.740135,11.0,19.0,US,USA,327167434.0,38.1
13340,US,2020-03-14,WA,572,WA-US,568.0,4.0,1,1.0,0.0,...,0.0,Washington,47.751076,-120.740135,12.0,20.0,US,USA,327167434.0,38.1


In [29]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
13326,US,2020-03-10,VA,7,VA-US,2.0,5.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,16.0,US,USA,327167434.0,38.1
13327,US,2020-03-11,VA,9,VA-US,7.0,2.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,17.0,US,USA,327167434.0,38.1
13328,US,2020-03-12,VA,17,VA-US,9.0,8.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,NaN,18.0,US,USA,327167434.0,38.1
13329,US,2020-03-13,VA,30,VA-US,17.0,13.0,0,0.0,0.0,...,0.0,Virginia,37.926868,-78.024902,1.0,19.0,US,USA,327167434.0,38.1
13330,US,2020-03-14,VA,41,VA-US,30.0,11.0,0,0.0,0.0,...,1.0,Virginia,37.926868,-78.024902,2.0,20.0,US,USA,327167434.0,38.1


In [30]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age
13096,US,2020-03-10,CA,144,CA-US,101.0,43.0,2,2.0,0.0,...,1.0,California,36.778259,-119.417931,9.0,16.0,US,USA,327167434.0,38.1
13097,US,2020-03-11,CA,177,CA-US,144.0,33.0,2,2.0,0.0,...,1.0,California,36.778259,-119.417931,10.0,17.0,US,USA,327167434.0,38.1
13098,US,2020-03-12,CA,221,CA-US,177.0,44.0,6,2.0,4.0,...,1.0,California,36.778259,-119.417931,11.0,18.0,US,USA,327167434.0,38.1
13099,US,2020-03-13,CA,282,CA-US,221.0,61.0,6,6.0,0.0,...,0.0,California,36.778259,-119.417931,12.0,19.0,US,USA,327167434.0,38.1
13100,US,2020-03-14,CA,340,CA-US,282.0,58.0,6,6.0,0.0,...,1.0,California,36.778259,-119.417931,13.0,20.0,US,USA,327167434.0,38.1


## QA of Data 

In [ ]:
# This indicates a shift in the data
df.loc[df['daily_diff_cases'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['province_state'].value_counts()

In [ ]:
df_final.loc[df_final['daily_diff_cases'] < 0]

In [ ]:
df_final.loc[(df_final['daily_diff_cases'] < 0) &
      (df_final['province_state'] == 'CA')]

In [ ]:
df_final.loc[df_final['country_region'] == 'China'].head(200).sort_values('rank')

In [ ]:
df_final.country_region.value_counts()